In [1]:
import pandas as pd
import numpy as np
from riskmatrix.alpha.alpha101 import compute_alpha101

In [2]:
FILTER_SYMBOLS = ['BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'XRPUSDT', 'DOGEUSDT', 'BNBUSDT', 'ADAUSDT', '1000SHIBUSDT', 'MATICUSDT', 'AVAXUSDT', 'ETCUSDT', 'LTCUSDT', 'LINKUSDT', 'DOTUSDT', 'FTMUSDT', '1000PEPEUSDT', 'GMTUSDT', 'BCHUSDT', 'FILUSDT', 'SANDUSDT', 'NEARUSDT', 'EOSUSDT', 'AXSUSDT', 'GALAUSDT', 'APEUSDT', 'ATOMUSDT', 'MANAUSDT', 'APTUSDT', 'OPUSDT', 'TRXUSDT', 'DYDXUSDT', 'ORDIUSDT', 'CHZUSDT', 'TRBUSDT', 'WIFUSDT', 'CRVUSDT', 'ARBUSDT', 'UNIUSDT', 'SUIUSDT', 'WLDUSDT', 'AAVEUSDT', 'WAVESUSDT', 'PEOPLEUSDT', 'SUSHIUSDT', 'MASKUSDT', 'RUNEUSDT', 'XLMUSDT', 'THETAUSDT', 'INJUSDT', 'LRCUSDT', 'CFXUSDT', 'VETUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ZILUSDT', 'SXPUSDT', 'XTZUSDT', 'GRTUSDT', 'LINAUSDT', 'MKRUSDT', 'TIAUSDT', 'STORJUSDT', 'UNFIUSDT', '1000BONKUSDT', 'NOTUSDT', 'ENJUSDT', 'NEOUSDT', 'EGLDUSDT', 'FETUSDT', '1INCHUSDT', '1000LUNCUSDT', 'STXUSDT', 'COMPUSDT', 'YFIUSDT', 'ONEUSDT', 'KNCUSDT', 'LDOUSDT', '1000SATSUSDT', 'SNXUSDT', 'ZECUSDT', 'ENSUSDT', 'OMGUSDT', 'BOMEUSDT', 'SEIUSDT', 'MTLUSDT', 'BLZUSDT', 'REEFUSDT', 'CHRUSDT', 'BAKEUSDT', 'KAVAUSDT', '1000FLOKIUSDT', 'ENAUSDT', 'RSRUSDT', 'XMRUSDT', 'RNDRUSDT', 'RLCUSDT', 'OGNUSDT', 'JASMYUSDT', 'BELUSDT', 'OCEANUSDT']

In [3]:
def compute_past_close_return(df, N):
    for i in range(N):
        df[f'past_close_return_{i+1}'] = df.groupby('symbol')['close'].transform(lambda x: (x / x.shift(i+1) - 1) * 100)
    df['returns'] = df['past_close_return_1']
    return df

In [4]:
def read_parquet(filename, filter_symbols):
    # read parquet file with pandas
    df = pd.read_parquet(filename)
    df['vwap'] = df['quote_volume']/df['volume']
    # filter symbols ending in USDT
    df = df[df['symbol'].str.endswith('USDT')]
    # FILTER_SYMBOLS = df[['symbol', 'quote_volume']].groupby('symbol').sum().sort_values('quote_volume', ascending=False).index.to_list()[:100]
    df = df[df['symbol'].isin(filter_symbols)]
    df.sort_values(['symbol', 'open_time'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df


In [5]:
df = read_parquet('../scratch/dp/data/all_data_1d.parquet', FILTER_SYMBOLS)
df = compute_past_close_return(df, 10)
df = compute_alpha101(df, open='open', high='high', low='low', close='close', volume='volume', returns='returns', vwap='vwap')

/home/eric/workspace/riskmatrix/riskmatrix/alpha/alpha101.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inner[self.returns < 0] = stddev(self.returns, 20)
/home/eric/workspace/riskmatrix/riskmatrix/alpha/alpha101.py:170: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/home/eric/workspace/riskmatrix/riskmatrix/alpha/alpha101.py:171: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
/home/eric/workspace/riskmatrix/riskmatrix/alpha/alpha101.py:170: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. 

In [6]:
df

,symbol,open_time,open,high,low,close,volume,close_time,quote_volume,count,...,alpha085,alpha086,alpha088,alpha092,alpha094,alpha095,alpha096,alpha098,alpha099,alpha101
0,BTCUSDT,1609459200000,28948.19,29668.86,28627.12,29337.160000,210716.398,1609545599999,6.157505e+09,1511793,...,NaN,0,0.060584,NaN,NaN,False,NaN,-0.608339,0,0.373385
1,BTCUSDT,1609545600000,29337.15,33480.00,28958.24,32199.910000,545541.080,1609631999999,1.712294e+10,3514545,...,NaN,0,0.024234,NaN,NaN,False,NaN,-0.608339,0,0.633107
2,BTCUSDT,1609632000000,32198.41,34832.25,32000.02,33054.530000,487486.989,1609718399999,1.638911e+10,3325307,...,NaN,0,0.098004,NaN,NaN,False,NaN,-0.608339,0,0.302278
3,BTCUSDT,1609718400000,33053.93,33653.56,27800.00,NaN,506128.023,1609804799999,1.587920e+10,3470951,...,NaN,0,0.098004,NaN,NaN,False,NaN,-0.608339,0,NaN
4,BTCUSDT,1609804800000,32029.55,34500.00,29900.00,33999.520000,433720.833,1609891199999,1.395935e+10,3077140,...,NaN,0,0.065574,NaN,NaN,False,NaN,-0.608339,0,0.428254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,BTCUSDT,1730246400000,72739.50,72984.90,71444.20,1.971861,227108.808,1730332799999,1.639468e+10,3249471,...,0.579369,0,0.878118,7.0,-0.741925,True,-13.0,-0.743763,-1,-47.210671
1399,BTCUSDT,1730332800000,72349.00,72720.30,69705.00,2.028995,256485.328,1730419199999,1.826086e+10,3578458,...,0.551265,0,0.949394,7.0,-0.811063,True,-13.0,-0.821098,-1,-23.993283
1400,BTCUSDT,1730419200000,70321.90,71650.00,68822.50,2.064053,352591.088,1730505599999,2.466970e+10,4618190,...,0.584569,0,0.981468,7.0,-0.739224,True,-13.0,-0.834996,-1,-24.869960
1401,BTCUSDT,1730505600000,69495.40,69913.10,69022.80,2.056389,79314.105,1730591999999,5.511280e+09,1401664,...,0.648565,0,0.992872,7.0,-0.708333,True,-13.0,-0.786885,-1,-78.056010
